In [1]:
import math
import numpy as np

In [2]:
# Link lengths and shoulder height (same as in forward_kinematics_points)
l1 = 6.1   # Shoulder → elbow
l2 = 11.5   # Elbow → end effector
h  = 6.8   # Shoulder height above ground

def inverse_kinematics_point(x, y, z, elbow_up=False):

    # 1) base yaw:
    phi = math.atan2(y, x)
    theta1_deg = math.degrees(phi + math.pi/2) # map to robot angles

    # 2) project into the shoulder's pitch plane:
    r = math.hypot(x, y) # horizontal distance from shoulder
    dz = z - h # vertical offset from shoulder

    # 3) two‑link planar IK:
    D = (r**2 + dz**2 - l1**2 - l2**2) / (2*l1*l2)
    if abs(D) > 1.0:
        raise ValueError(f"Point ({x}, {y}, {z}) is out of reach (|D|={D:.3f}>1).") # limitation of the arm length

    # 4) elbow angle:
    if elbow_up:
        theta3_rad = math.atan2(+math.sqrt(1-D**2), D)
    else:
        theta3_rad = math.atan2(-math.sqrt(1-D**2), D)

    # 5) shoulder pitch:
    alpha = math.atan2(dz, r)
    beta = math.atan2(l2 * math.sin(theta3_rad), l1 + l2 * math.cos(theta3_rad))
    theta2_rad = alpha - beta

    # 6) map to robot angle conventions:
    theta2_deg = math.degrees(math.pi - theta2_rad)
    theta3_deg = math.degrees(theta3_rad + math.pi)

    return theta1_deg, theta2_deg, theta3_deg

In [3]:
def forward_kinematics_points(theta1_deg, theta2_deg, theta3_deg):
    theta1 = math.radians(theta1_deg) - np.pi/2
    theta2 = np.pi - math.radians(theta2_deg)
    theta3 = -(np.pi - math.radians(theta3_deg))

    l1 = 6.1   # Shoulder to elbow
    l2 = 11.5   # Elbow to end effector
    h = 6.8    # Shoulder height from ground

    x0, y0, z0 = 0, 0, 0
    x1, y1, z1 = 0, 0, h

    x2 = x1 + l1 * math.cos(theta2) * math.cos(theta1)
    y2 = y1 + l1 * math.cos(theta2) * math.sin(theta1)
    z2 = z1 + l1 * math.sin(theta2)

    total_angle = theta2 + theta3
    x3 = x2 + l2 * math.cos(total_angle) * math.cos(theta1)
    y3 = y2 + l2 * math.cos(total_angle) * math.sin(theta1)
    z3 = z2 + l2 * math.sin(total_angle)

    return [(x0, y0, z0), (x1, y1, z1), (x2, y2, z2), (x3, y3, z3)]

In [4]:
point = np.array([4.5, 5, 2])
result = inverse_kinematics_point(point[0], point[1], point[2], elbow_up=False)

fk = forward_kinematics_points(result[0], result[1], result[2])
compare = fk[3]
new_diff = np.round(point - compare, 1)

print(new_diff)

[0. 0. 0.]


In [5]:
import itertools

# initiate the ranges of possible integer values of x, y and z
range_y = range(-5, 6)   # -5 to 5
range_x = range(6, 15)   # 6 to 14
range_z = range(0, 7)    # 0 to 6

# create every combination of the previous:
combinations = list(itertools.product(range_y, range_x, range_z))

space = np.array(combinations)

In [12]:
diffs = np.empty((1,))

# apply inverse kinematics followed by direct kinematics to every points, to check function complementarity:
for point in space:
    result = inverse_kinematics_point(point[0], point[1], point[2], elbow_up=True)
    fk = forward_kinematics_points(result[0], result[1], result[2])
    compare = fk[3]
    new_diff = np.round(np.abs(point - compare), 3)

    diffs = np.append(diffs,new_diff)

In [13]:
print(diffs)

[0. 0. 0. ... 0. 0. 0.]
